In [1]:
#FP树:Frequent Pattern Tree

#树节点
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {}  #存放子节点的字典
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print '  '*ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind+1)
            
#测试一下
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.disp()

print '*******************'

rootNode.children['phoenix'] = treeNode('phoenix',3,None)
rootNode.disp()

   pyramid   9
     eye   13
*******************
   pyramid   9
     eye   13
     phoenix   3


In [10]:
#创建FP树
#dataset为字典{frozenset([事务i]):频数} ,minSup:最小支持度（用于过滤非频繁集）
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {} #头指针表,结构{z:5}--->{z:[5,nodeLink]}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance,统计每个元素项的个数
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans] #累计
    for k in headerTable.keys():  #remove items not meeting minSup,删除频数很小的元素项
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys()) #频繁项集
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: 
        return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree,空集开始
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        #第二次遍历，一个事务一个事务的处理,首先进行一个事务的元素项排序
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0] #{频繁项：频数} 为了排序
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted
            (localD.items(), key=lambda p: p[1], reverse=True)] #从大到小排序得[a,b,c...]
            #将这个事务添加到FP树
            updateTree(orderedItems, retTree, headerTable, count) 
            #populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table
#对排序的频繁项集，创建FP树
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 改变头指针表中的nodeLink
            headerTable[items[0]][1] = inTree.children[items[0]]
        else: 
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]]) 
    if len(items) > 1:#call updateTree() with remaining ordered items
        #每次调用，去掉index = 0
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode #从头指针表的nodeLink开始，一直沿着nodeLink直到链表尾部
    

#测试一些函数
localD = {'a':5,'b':3,'c':6}
sortD = sorted(localD.items(), key=lambda p: p[1], reverse=True) #从大到小排序
print sortD
orderedItems = [v[0] for v in sortD] #从大到小排序
print orderedItems

l = [1,2,3,4]
def sum(l):
    if len(l)>0:
        print l[0]
        sum(l[1::])
sum(l)

[('c', 6), ('a', 5), ('b', 3)]
['c', 'a', 'b']
1
2
3
4


In [12]:
#构建数据集
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
myFPtree, myHeaderTab = createTree(initSet,minSup=3)
myFPtree.disp()

print myHeaderTab


   Null Set   1
     x   1
       s   1
         r   1
     z   5
       x   3
         y   3
           s   2
             t   2
           r   1
             t   1
       r   1
{'s': [3, <__main__.treeNode instance at 0x00000000059A1388>], 'r': [3, <__main__.treeNode instance at 0x00000000059A15C8>], 't': [3, <__main__.treeNode instance at 0x00000000059A16C8>], 'y': [3, <__main__.treeNode instance at 0x00000000059A1688>], 'x': [4, <__main__.treeNode instance at 0x00000000059A11C8>], 'z': [5, <__main__.treeNode instance at 0x00000000059A1548>]}


In [13]:
#接下来，从一个FP树中挖掘频繁项集

#1.抽取条件模式基（conditional pattern base）:所查找元素项为结尾的路径集合
#前缀路径：介于所查找元素项与树根节点之间的所有内容   P231

#从所查元素项leafNode，迭代上溯父节点parent，并且保存名称name
#x->y->z得到[z,y,x]
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count #[1:]不包含自身,
            # 而且记录结尾元素项(所查元素项)的个数count
        treeNode = treeNode.nodeLink #下一个链接节点
    return condPats

condPats1 = findPrefixPath('x',myHeaderTab['x'][1])
print condPats1

condPats2 = findPrefixPath('z',myHeaderTab['z'][1])
print condPats2

condPats3 = findPrefixPath('r',myHeaderTab['r'][1])
print condPats3




{frozenset(['z']): 3}
{}
{frozenset(['x', 's']): 1, frozenset(['z']): 1, frozenset(['y', 'x', 'z']): 1}


In [20]:
#2.创建条件FP树
#对于每一个频繁项，都要创建一个条件FP树

def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1])]#(sort header table)
    for basePat in bigL:  #start from bottom of header table #遍历，对每一个频繁项创建FP树
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print 'finalFrequent Item: ',newFreqSet    #append to set
        freqItemList.append(newFreqSet) #添加到频繁项集
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1]) #创建条件基
        #print 'condPattBases :',basePat, condPattBases
        #2. construct cond FP-tree from cond. pattern base
        myCondTree, myHead = createTree(condPattBases, minSup) #根据条件基创建条件FP树
        #print 'head from conditional tree: ', myHead
        if myHead != None: #3. mine cond. FP-tree
            print 'conditional tree for: ',newFreqSet
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
            
freqItems = [] #存储频繁项集
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)
            
            

conditional tree for:  set(['s'])
   Null Set   1
     x   3
conditional tree for:  set(['y'])
   Null Set   1
     x   3
       z   3
conditional tree for:  set(['y', 'z'])
   Null Set   1
     x   3
conditional tree for:  set(['t'])
   Null Set   1
     y   3
       x   3
         z   3
conditional tree for:  set(['z', 't'])
   Null Set   1
     y   3
       x   3
conditional tree for:  set(['x', 'z', 't'])
   Null Set   1
     y   3
conditional tree for:  set(['x', 't'])
   Null Set   1
     y   3
conditional tree for:  set(['x'])
   Null Set   1
     z   3


In [15]:
print freqItems

[set(['s']), set(['x', 's']), set(['r']), set(['y']), set(['y', 'z']), set(['y', 'x', 'z']), set(['y', 'x']), set(['t']), set(['y', 't']), set(['x', 't']), set(['y', 'x', 't']), set(['z', 't']), set(['y', 'z', 't']), set(['x', 'z', 't']), set(['y', 'x', 'z', 't']), set(['x']), set(['x', 'z']), set(['z'])]


In [19]:
#展示fp-growth算法速度
#100万条记录文件
import time
parsedDat = [line.split() for line in open('fp-growth/kosarak.dat').readlines()]
initSet1 = createInitSet(parsedDat)
time1 = time.time()
myFPtree1,myHeaderTab1 = createTree(initSet1,100000) #最小频数为100000
time2 = time.time()
print 'creating the fp-tree cost ',time2-time1,'s'
myFreqList = []
mineTree(myFPtree1,myHeaderTab1,100000,set([]),myFreqList)
print len(myFreqList)
time3 = time.time()
print "getting the frequent sets cost ",(time3-time1),'s'


creating the fp-tree cost  15.4380002022 s
9
getting the frequent sets cost  15.4620001316 s
